#Improving Computer Vision Accuracy using Convolutions


前回のレッスンでは、入力層（データの形をした）、出力層（希望する出力の形をした）、そして隠れ層の3つの層を含むディープニューラルネットワーク（DNN）を使ってファッション認識を行う方法を見ました。隠蔽層の大きさ、訓練エポック数などが最終的な精度に与える影響を実験しました。

便宜上、ここにコード全体をもう一度示します。これを実行して、最後にプリントアウトされたテスト精度をメモしておいてください。

In [ ]:
import tensorflow as tf
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images / 255.0
test_images=test_images / 255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=5)

test_loss = model.evaluate(test_images, test_labels)

あなたの精度はおそらくトレーニングで約89%、検証で約87%です...悪くないですね...でも、どうやってそれをさらに良くするのでしょうか？一つの方法は畳み込み（Convolutions）と呼ばれるものを使うことです。ここでは畳み込みの詳細については触れませんが、究極のコンセプトは、画像の内容を絞り込んで、特定の明確な細部に焦点を当てるというものです。

フィルターを使った画像処理をしたことがある人ならば (https://en.wikipedia.org/wiki/Kernel_(image_processing) のように)、 畳み込みは非常に見慣れたものになるでしょう。

要するに、配列 (通常は 3x3 や 5x5) を受け取り、それを画像に渡します。その行列の中の式に基づいて基礎となるピクセルを変更することで、エッジ検出のようなことができます。例えば、上のリンクを見てみると、3x3はエッジ検出用に定義されており、真ん中のセルが8で、隣り合うセルはすべて-1となっています。 この場合、各ピクセルの値に8をかけ、隣り合うセルの値を差し引きます。これを各画素ごとに行うと、エッジが強調された新しい画像ができあがります。

![元画像](https://upload.wikimedia.org/wikipedia/commons/5/50/Vd-Orig.png)
![変換後](https://upload.wikimedia.org/wikipedia/commons/6/6d/Vd-Edge3.png)

これはコンピュータビジョンに最適です。このようにハイライトされた特徴によって、あるアイテムを別のアイテムと区別することができ、必要とされる情報量はずっと少なくて済みます。なぜならハイライトされた部分だけ学習させればいいわけですので。

これが畳み込みニューラルネットワークのコンセプトです。全結合層の前にいくつかの畳み込み層を追加すると、全結合層に送られる情報はより集中され、より正確になる可能性があります。

以下のコードを実行してみてください -- これは先ほどと同じニューラルネットワークですが、今回は畳み込み層を最初に追加しています。時間はかかりますが、精度への影響を見てください。

In [ ]:
import tensorflow as tf
print(tf.__version__)
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(training_images, training_labels, epochs=5)
test_loss = model.evaluate(test_images, test_labels)


トレーニングデータでは93％、検証データでは91％くらいまで上がっているのではないでしょうか。

これは重要な一歩です。

より多くのエポックで実行してみてください - 例えば、約20のエポックで、結果をみてください。結果はとても良いように見えるかもしれませんが、後述する「過学習」と呼ばれるものが原因で、検証結果は実際には下がっているかもしれません。

(簡単に言えば、「過学習」とは、ネットワークが訓練セットからのデータをよく学習しているにもかかわらず、そのデータだけに特化しすぎていて、結果的に違う（見たことのない）データを判断することができなくなることです。例えば、人生の中で赤い靴しか見たことがなかった場合、赤い靴を見たときにそれを識別するのは非常に得意ですが、青いスエードの靴を見たときには混乱するかもしれません。)

それから、もう一度コードを見て、畳み込みがどのように構築されたかを一歩一歩見てください。

ステップ1はデータを収集することです。ここで、学習データの形を変える必要があるという点で、少し変化があることに気づくでしょう。これは、最初の畳み込みではすべてを含む1つのテンソルを想定しているからです。そのため、60,000個の28x28x1のリストの代わりに、60,000x28x28x1の1つの4次元リストを作成し、テスト画像についても同様です。このようにしないと、畳み込み層が形状を認識しないため、トレーニング時にエラーが出てしまいます。


```
import tensorflow as tf
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0
```



次にモデルを定義します。ここでは、一番上の入力レイヤーの代わりに Convolution を追加します。パラメータは以下の通りです。

1. 生成するフィルター（カーネル）の数。純粋に任意ですが、32 くらいから始めるのが良いでしょう。
2. フィルターのサイズ（この場合は3x3）。
3. 使用する活性化関数 -- ここでは relu を使用します。これは x>0 のときに x を返し、そうでなければ 0 を返すことと同等のものです。
4. 最初のレイヤーで利用する畳み込みは、入力データの形状を指定します。

コンボリューションで強調された特徴の内容を維持しつつ、画像を圧縮するように設計されたMaxPoolingレイヤーでコンボリューションを追いかけます。MaxPooling に (2,2) を指定すると、画像のサイズが 4 分の 1 になります。ここではあまり詳しく説明しませんが、ピクセルの2x2配列を作成し、一番大きいものを選び、4ピクセルを1ピクセルに変換します。 これを画像全体で繰り返すことで、水平方向のピクセル数を半分に、垂直方向のピクセル数を半分にし画像を25%に縮小します。

ネットワークの大きさや形を見るには model.summary() を呼び出すことができ、MaxPooling レイヤーを重ねるごとに画像サイズがこのように縮小されていることがわかると思います。

```
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
```



さらに一つの畳込み（Convolution）を追加します。
```
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2)
```



出力を平坦化します。これ以降は畳み込みを利用していないDNNと同じ構造になります。

```
  tf.keras.layers.Flatten(),
```

全結合層を１２８層、出力層を１０層としたのは、畳み込みを利用しない場合と同じです。

```
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
```

ここでモデルをコンパイルし、fitメソッドを呼び出して学習を行ったあと、テストセットを利用して損失と精度を評価します。



```
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=5)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)


# Visualizing the Convolutions and Pooling
このコードは、畳み込みの状況をグラフィカルに表示してくれます。print (test_labels[:100])は、テストセットの最初の100個のラベルを示しています。インデックス0、インデックス23、インデックス28のラベルはすべて同じ値(9)であることがわかります。これらはすべて靴です。それぞれに畳み込みを実行した結果を見てみましょう。畳み込み後のDNNがそのデータで学習しているときには、より少ない量のデータで学習しているので、この畳み込みとプールの組み合わせの出力から靴の間の共通性を見つけているのかもしれません。

In [ ]:
print(test_labels[:100])

In [ ]:
import matplotlib.pyplot as plt
f, axarr = plt.subplots(3,4)
FIRST_IMAGE=0
SECOND_IMAGE=23
THIRD_IMAGE=28
CONVOLUTION_NUMBER = 1
from tensorflow.keras import models
layer_outputs = [layer.output for layer in model.layers]
activation_model = tf.keras.models.Model(inputs = model.input, outputs = layer_outputs)
for x in range(0,4):
  f1 = activation_model.predict(test_images[FIRST_IMAGE].reshape(1, 28, 28, 1))[x]
  axarr[0,x].imshow(f1[0, : , :, CONVOLUTION_NUMBER], cmap='inferno')
  axarr[0,x].grid(False)
  f2 = activation_model.predict(test_images[SECOND_IMAGE].reshape(1, 28, 28, 1))[x]
  axarr[1,x].imshow(f2[0, : , :, CONVOLUTION_NUMBER], cmap='inferno')
  axarr[1,x].grid(False)
  f3 = activation_model.predict(test_images[THIRD_IMAGE].reshape(1, 28, 28, 1))[x]
  axarr[2,x].imshow(f3[0, : , :, CONVOLUTION_NUMBER], cmap='inferno')
  axarr[2,x].grid(False)

EXERCISES

1. 畳み込みを編集してみてください。32を16または64に変更してください。これは精度やトレーニング時間にどのような影響を与えるでしょうか。

2. 最後の畳み込みを削除します．これは精度やトレーニング時間にどのような影響を与えますか?

3. Convolution を追加してみてはどうでしょうか? これはどのような影響があると思いますか？実験してみてください．

4. 最初のConvolutions以外のすべてのConvolutionsを削除します。これはどのような影響があると思いますか？実験してみてください。

5. 前回のレッスンでは、損失関数をチェックして、一定の量に達したら学習をキャンセルするコールバックを実装しました。ここでそれを実装できるかどうか見てみてください。

In [ ]:
import tensorflow as tf
print(tf.__version__)
mnist = tf.keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=10)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)